5.	Address the challenges of transfer learning when dealing with small datasets.
a.	Choose a small dataset and demonstrate the effectiveness of transfer learning compared to training a CNN from scratch.
b.	Experiment with data augmentation techniques and analyze their impact on model generalization.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Input, Dense, Conv2D, MaxPooling2D

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

def CNN():
    Model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), padding='same', activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(256, (3,3), padding='same', activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])

    Model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return Model

model = CNN()
model.fit(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))
CNN_loss, CNN_accuracy = model.evaluate(x_test, y_test)
print(f"CNN Loss: {CNN_loss}")
print(f"CNN Accuracy: {CNN_accuracy}")

C:\Users\Sakuntala\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 17ms/step - accuracy: 0.3616 - loss: 1.7259 - val_accuracy: 0.5894 - val_loss: 1.1561
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.6373 - loss: 1.0195 - val_accuracy: 0.6863 - val_loss: 0.8839
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.7273 - loss: 0.7755 - val_accuracy: 0.7261 - val_loss: 0.8015
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 23ms/step - accuracy: 0.7779 - loss: 0.6372 - val_accuracy: 0.7333 - val_loss: 0.7943
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 24ms/step - accuracy: 0.8179 - loss: 0.5233 - val_accuracy: 0.7314 - val_loss: 0.7833
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7387 - loss: 0.7723
CNN Loss: 0.7832694053649902
CNN Accuracy: 0.7314000129699707


In [4]:
from tensorflow.keras.models import Model

def TransferLearning():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

    for layers in base_model.layers:
        layers.trainable = False

    layer = base_model.output
    layer = Flatten()(layer)
    layer = Dense(1024, activation='relu')(layer)
    layer = Dense(10, activation='softmax')(layer)

    model = Model(inputs=base_model.input, outputs = layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = TransferLearning()
model.fit(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))
Transfer_loss, Transfer_accuracy = model.evaluate(x_test, y_test)
print(f"Transfer Learning Loss: {Transfer_loss}")
print(f"Transfer Learning Accuracy: {Transfer_accuracy}")

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 239s 152ms/step - accuracy: 0.4927 - loss: 1.4506 - val_accuracy: 0.5655 - val_loss: 1.2336
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 255s 163ms/step - accuracy: 0.5979 - loss: 1.1389 - val_accuracy: 0.5945 - val_loss: 1.1609
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 167ms/step - accuracy: 0.6252 - loss: 1.0645 - val_accuracy: 0.6026 - val_loss: 1.1321
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 309s 197ms/step - accuracy: 0.6492 - loss: 0.9973 - val_accuracy: 0.6062 - val_loss: 1.1307
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 325s 208ms/step - accuracy: 0.6687 - loss: 0.9377 - val_accuracy: 0.6148 - val_loss: 1.1083
313/313 ━━━━━━━━━━━━━━━━━━━━ 56s 179ms/step - accuracy: 0.6173 - loss: 1.1098
Transfer Learning Loss: 1.1083109378814697
Transfer Learning Accuracy: 0.614799976348877


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataGeneration = ImageDataGenerator(
    rotation_range = 16,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

CNN_model = CNN()
Transfer_model = TransferLearning()
CNN_model.fit(dataGeneration.flow( x_train, y_train, batch_size = 16),
              epochs = 5, validation_data=(x_test, y_test))
Transfer_model.fit(dataGeneration.flow( x_train, y_train, batch_size = 16),
              epochs = 5, validation_data=(x_test, y_test))

CNN_loss_aug, CNN_accuracy_aug = CNN_model.evaluate(x_test, y_test)
Transfer_loss_aug, Transfer_accuracy_aug = Transfer_model.evaluate(x_test, y_test)

print(f"CNN Loss Without Augmentation :- {CNN_loss}")
print(f"CNN Accuracy Without Augmentation :- {CNN_accuracy}")
print(f"CNN Loss With Augmentation :- {CNN_loss_aug}")
print(f"CNN Accuracy With Augmentation :- {CNN_accuracy_aug}")
print(f"Transfer Learning Loss Without Augmentation :- {Transfer_loss}")
print(f"Transfer Learning Accuracy Without Augmentation :- {Transfer_accuracy}")
print(f"Transfer Learning Loss With Augmentation :- {Transfer_loss_aug}")
print(f"Transfer Learning Accuracy With Augmentation :- {Transfer_accuracy_aug}")

C:\Users\Sakuntala\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5
   5/3125 ━━━━━━━━━━━━━━━━━━━━ 1:23 27ms/step - accuracy: 0.1079 - loss: 2.3222

C:\Users\Sakuntala\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 75s 23ms/step - accuracy: 0.2887 - loss: 1.8844 - val_accuracy: 0.5247 - val_loss: 1.3359
Epoch 2/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 79s 25ms/step - accuracy: 0.5091 - loss: 1.3528 - val_accuracy: 0.5565 - val_loss: 1.2570
Epoch 3/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 69s 22ms/step - accuracy: 0.5742 - loss: 1.1820 - val_accuracy: 0.6001 - val_loss: 1.1716
Epoch 4/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 73s 23ms/step - accuracy: 0.6102 - loss: 1.0927 - val_accuracy: 0.6564 - val_loss: 0.9650
Epoch 5/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 73s 23ms/step - accuracy: 0.6356 - loss: 1.0268 - val_accuracy: 0.6671 - val_loss: 0.9632
Epoch 1/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 368s 117ms/step - accuracy: 0.4260 - loss: 1.6063 - val_accuracy: 0.5368 - val_loss: 1.3142
Epoch 2/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 366s 117ms/step - accuracy: 0.5169 - loss: 1.3655 - val_accuracy: 0.5555 - val_loss: 1.2448
Epoch 3/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 396s 127ms/step - accuracy: 0.5331 - loss: 1.315